# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## Important Note

This assignment requires you to have a version of Tweepy that is at least version 4. The latest version is 4.10 as I write this. Critically, this version of Tweepy is *not* on the upgrade path from Version 3, so you will not be able to simply upgrade the package if you are on Version 3. Instead you will need to explicitly install version 4, which you can do with a command like this: `pip install "tweepy>=4"`. You will also be using Version 2 of the Twitter API for this assignment. 

Run the below cell. If your version of Tweepy begins with a "4", then you should be good to go. If it begins with a "3" then run the following command, found [here](https://stackoverflow.com/questions/5226311/installing-specific-package-version-with-pip), at the command line or in a cell: `pip install -Iv tweepy==4.9`. (You may want to update that version number if Tweepy has moved on past 4.9. 

In [1]:
# let's install the appropriate version of tweepy
!pip install "tweepy>=4"

In [2]:
# verify tweepy version
!pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: requests-oauthlib, requests, oauthlib
Required-by: 


# Twitter API Pull

In [3]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter


In [4]:
# Use this cell for any import statements you add
import random
import pandas as pd

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [5]:
# contents of api_keys.py will be hidden from repository to maintain secrecy
# api_keys.py was created manually with API keys copied over
from api_keys import api_key, api_key_secret, bearer_token

In [6]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [7]:
# this identifies the twitter handle we'll be basing our pulls from
handle = "37chandler"
user_obj = client.get_user(username=handle)

# see https://docs.tweepy.org/en/v4.0.1/client.html
# client.get_users_followers retrieves follower data of specified handle
followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)



Rate limit exceeded. Sleeping for 547 seconds.


Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [8]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    # let's separate the public_metrics into following and follower counts
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Dave Renn lists 'None' as their location.
 Following: 43, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 220, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 219, Followers: 61.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1591, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2819, Followers: 424.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2795, Followers: 3562.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2801, Followers: 3338.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers. 

In [9]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14185, 'following_count': 2221, 'tweet_count': 56119, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [10]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [11]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for username we have 37chandler
for verified we have False
for location we have MN
for name we have John Chandler
for id we have 33029025
for created_at we have 2009-04-18 22:08:22+00:00
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 997, 'listed_count': 3}


Now a few questions for you about the user object.

Q: How many fields are being returned in the `response` object? 

A: We have nine fields being returned in the `response` object. One of the fields ("public_metrics") returns four subfields.

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: Yes, one of the fields ("profile_image_url") returns a url for a profile image.

---

Q: How many friends, followers, and tweets does this user have? 

A: This user has 589 friends, 193 followers, and 997 tweets.


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [12]:
# see https://docs.tweepy.org/en/v4.0.1/client.html
# this retrieves tweets from specified handle (user_obj.data.id)
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    # the following indicates that when the idx iteration is greater than 10 -> stop
    if idx > 10 :
        break

1569760631548690437
RT @dtmooreeditor: So there's a particular quirk of English grammar that I've always found quite endearing: the exocentric verb-noun compou…

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent bo

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

In [13]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=25, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break
        



## Pulling Twitter Data for Your Artists

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [14]:
artists = dict()

# this is a for loop to cycle through both MCR and Missy Elliot's twitter handles
for handle in ['MCRofficial','MissyElliott'] : 
    # client.get_user will get info about the handles
    # in this case, we're indicating the username to be the handles specified
    # the fields we'll retrieve will be info in public_metrics
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    # this specifies for an artist with a specific handle to generate their id, handle, and follower count
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    
# this generates a print statement using info from artists
for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 24.61 hours to pull all 1476574 followers for MCRofficial. 
It would take 117.18 hours to pull all 7030671 followers for MissyElliott. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist. 

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [15]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In addition to creating a file that only has follower IDs in it, you will create a file that includes user data. From the response object please extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 

One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

### Artist 1: My Chemical Romance (@MCRofficial)

In [16]:
# note the artists' twitter handles
handles = ['MCRofficial']
handle_followers = []

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_obj = client.get_user(username=handle) # this will specify the handles
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    # this produces a print statement indicating artist
    print(f'Pulling followers for {handle}.')
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for followers in tweepy.Paginator(client.get_users_followers,
            user_obj.data.id,
            user_fields=["username","name","id","location","public_metrics","description"],
            max_results=1000,
            limit=100):
        print(followers.meta)
        
        for follower in followers.data:
            follower_row = {'id': follower.id,
                            'username': follower.username,
                            'name': follower.name,
                            'location': follower.location,
                            'follower_count': follower.public_metrics['followers_count'],
                            'following': follower.public_metrics['following_count'],
                            'description': follower.description
                           }
            user_data[handle].append(follower_row) # this appends all follower data
            followers_data[handle].append(follower.id) # this appends only follower ids
            
    # For each response object, extract the needed fields and store them in a dictionary or
    # data frame. 
    users_output_MCR = pd.DataFrame(user_data[handle], columns=['id','username',
                                                               'name', 'location',
                                                               'follower_count', 'following',
                                                               'description'])
    followers_output_MCR = pd.DataFrame(user_data[handle], columns=['id'])
    
    users_output_MCR.to_csv(f'twitter/{handle}_followers.txt', sep='\t', index=False)
    followers_output_MCR.to_csv(f'twitter/{handle}_follower_data.txt', sep='\t', index=False)
    
    # with help from Martin Zagari via slack, inserting planned sleep to help with rate limit        
    time.sleep(25+(10*random.random()))
            
    print(f'Completed pulling follower data for {handle}.')
        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(f'Time taken was {end_time - start_time}.')


Pulling followers for MCRofficial.
{'result_count': 1000, 'next_token': 'P2HJJPCTQ0P1GZZZ'}
{'result_count': 1000, 'next_token': 'MSV1DMPMTGOHGZZZ', 'previous_token': 'QN4F11SS5V6UEZZZ'}
{'result_count': 1000, 'next_token': 'G3962HEPB0OHGZZZ', 'previous_token': 'GM63TGVE2F7EEZZZ'}
{'result_count': 1000, 'next_token': 'TBVKGJB6QGO1GZZZ', 'previous_token': 'T8LBNSHKKV7EEZZZ'}
{'result_count': 1000, 'next_token': 'R1VFGLUUDKO1GZZZ', 'previous_token': '3JQRDH585F7UEZZZ'}


Rate limit exceeded. Sleeping for 896 seconds.


{'result_count': 1000, 'next_token': '6ARJUS71OCNHGZZZ', 'previous_token': '9S4E8MP3IB7UEZZZ'}
{'result_count': 1000, 'next_token': 'MS4RO0P11SNHGZZZ', 'previous_token': 'JHE80TQR7J8EEZZZ'}
{'result_count': 1000, 'next_token': 'E7G3LNG9BCN1GZZZ', 'previous_token': 'NPO9DB72U38EEZZZ'}
{'result_count': 1000, 'next_token': 'C0BF0G5LMKMHGZZZ', 'previous_token': 'OBD25OGNKN8UEZZZ'}
{'result_count': 1000, 'next_token': 'AIMRQ79O04MHGZZZ', 'previous_token': '36QHFR349B9EEZZZ'}
{'result_count': 1000, 'next_token': '3USB01FUGOM1GZZZ', 'previous_token': 'T3JH9JO1VV9EEZZZ'}
{'result_count': 1000, 'next_token': '5G4S0A7V08M1GZZZ', 'previous_token': '0JC1HT8LF79UEZZZ'}
{'result_count': 1000, 'next_token': '8U6FHEHA7CLHGZZZ', 'previous_token': 'E5FPB8R0VN9UEZZZ'}
{'result_count': 1000, 'next_token': '2LL9NQ4S50L1GZZZ', 'previous_token': 'UTOV6PU1ONAEEZZZ'}
{'result_count': 1000, 'next_token': 'HGBQJBEGNCK1GZZZ', 'previous_token': 'PHL0BFT4QVAUEZZZ'}
{'result_count': 1000, 'next_token': 'RRI7VN9CLGJH

Rate limit exceeded. Sleeping for 893 seconds.


{'result_count': 1000, 'next_token': 'NPHU8PSDJKGHGZZZ', 'previous_token': '17034DIN8BEUEZZZ'}
{'result_count': 1000, 'next_token': 'B98V5QRBIKG1GZZZ', 'previous_token': '4OO1PAK4CBFEEZZZ'}
{'result_count': 1000, 'next_token': 'NE5VR6IGI8FHGZZZ', 'previous_token': 'QVG8JIKQDBFUEZZZ'}
{'result_count': 1000, 'next_token': 'AMJA0TROE4F1GZZZ', 'previous_token': 'CMURN5F3DNGEEZZZ'}
{'result_count': 1000, 'next_token': 'VG61LG827CEHGZZZ', 'previous_token': 'MSSDI4LEHVGUEZZZ'}
{'result_count': 1000, 'next_token': '52M6BCBAT4DHGZZZ', 'previous_token': 'TP98918CONHEEZZZ'}
{'result_count': 1000, 'next_token': 'IKU6ULFBNGD1GZZZ', 'previous_token': '248S8TSL2RIEEZZZ'}
{'result_count': 1000, 'next_token': '1KBG51CQN0CHGZZZ', 'previous_token': 'KO8LP9958FIUEZZZ'}
{'result_count': 1000, 'next_token': 'FH366QF7JKC1GZZZ', 'previous_token': '15NBUBDE8VJEEZZZ'}
{'result_count': 999, 'next_token': 'OHDRMQQHHKBHGZZZ', 'previous_token': 'J7SMR4D5CBJUEZZZ'}
{'result_count': 1000, 'next_token': 'KUH3I125DKB1G

...

{'result_count': 1000, 'next_token': '3KLAPAJNLK5HGZZZ', 'previous_token': 'HKC9NGJIA3QEEZZZ'}
{'result_count': 1000, 'next_token': 'QMS3I25MLC5HGZZZ', 'previous_token': '08HM2E49ABQEEZZZ'}
{'result_count': 1000, 'next_token': '8KKC76INL85HGZZZ', 'previous_token': '6260FVQAAJQEEZZZ'}
{'result_count': 1000, 'next_token': '835CN3PCL45HGZZZ', 'previous_token': 'C1RJN7L8ANQEEZZZ'}
{'result_count': 1000, 'next_token': 'TKHIRI1DL05HGZZZ', 'previous_token': '3HCUTREJARQEEZZZ'}
{'result_count': 1000, 'next_token': 'GBOIK8HDKS5HGZZZ', 'previous_token': 'MKA58GEIAVQEEZZZ'}
{'result_count': 1000, 'next_token': 'I99R1U1BKO5HGZZZ', 'previous_token': '3VOERTMIB3QEEZZZ'}
{'result_count': 1000, 'next_token': 'CK5IT08AKK5HGZZZ', 'previous_token': 'NBIFK56KB7QEEZZZ'}
{'result_count': 1000, 'next_token': 'DPTTULTRIG51GZZZ', 'previous_token': 'TIVRQ57MBBQEEZZZ'}
{'result_count': 1000, 'next_token': '0ESRTER53G4HGZZZ', 'previous_token': '55UNULADDJQUEZZZ'}
{'result_count': 999, 'next_token': 'OHLBU3B9IO3HG

Rate limit exceeded. Sleeping for 893 seconds.


{'result_count': 1000, 'next_token': '3DP7D6POCFVHEZZZ', 'previous_token': '9I7TGBQ4VFVEEZZZ'}
{'result_count': 1000, 'next_token': 'CCL2GQTQ3NV1EZZZ', 'previous_token': 'TC04L2AFJK0EGZZZ'}
{'result_count': 1000, 'next_token': 'F6N4235VFRUHEZZZ', 'previous_token': '3KV6J2AMS80UGZZZ'}
{'result_count': 1000, 'next_token': 'TNPK7Q88C3U1EZZZ', 'previous_token': 'E58A50A6G41EGZZZ'}
{'result_count': 1000, 'next_token': 'KJ4K6SI63FTHEZZZ', 'previous_token': 'BEO4GSIIK01UGZZZ'}
{'result_count': 1000, 'next_token': 'LU246MT2P3SHEZZZ', 'previous_token': '6D9F906MSG2EGZZZ'}
{'result_count': 1000, 'next_token': '06LCSJUU6JSHEZZZ', 'previous_token': '3H0MK5HF703EGZZZ'}
{'result_count': 1000, 'next_token': 'QOAP0KJNAVS1EZZZ', 'previous_token': '4E1PQ4PAPC3EGZZZ'}
{'result_count': 999, 'next_token': '10SM514S9VRHEZZZ', 'previous_token': 'ERKBCSDBL03UGZZZ'}
{'result_count': 1000, 'next_token': '00FINF74T7R1EZZZ', 'previous_token': 'FUTPHP5LM04EGZZZ'}
{'result_count': 1000, 'next_token': 'QV5E7K4C5NQHE

Rate limit exceeded. Sleeping for 892 seconds.


{'result_count': 1000, 'next_token': 'EGJKDQOHKJM1EZZZ', 'previous_token': '4QBSHO4MM48UGZZZ'}
{'result_count': 1000, 'next_token': 'TNTCB2A017LHEZZZ', 'previous_token': '9M7QT1RRBG9UGZZZ'}
{'result_count': 1000, 'next_token': 'V3MCB22CDBKHEZZZ', 'previous_token': '34NI3IV7UOAEGZZZ'}
{'result_count': 1000, 'next_token': 'TCK08SRO23K1EZZZ', 'previous_token': '68PRLVOOIOBEGZZZ'}
{'result_count': 1000, 'next_token': 'J79B4SKIC7JHEZZZ', 'previous_token': '9FR5AOMGTSBUGZZZ'}
Completed pulling follower data for MCRofficial.
Time taken was 1:45:43.055334.


In [17]:
# checking dataframes to ensure proper data pull
users_output_MCR.head()

,id,username,name,location,follower_count,following,description
0,1470235406348263425,chemiclly101,kai,None,0,66,i repost not so appropriate stuff a lot 🔞
1,1090096418826715136,debbiecola,debbiecola,None,0,12,
2,1562612060013268993,AshtonJaxx03,Ashton Jaxx Barnes,None,0,14,
3,1340336528355110912,yukuakiii,shutthefup,in gerard way,0,44,xia // xntj 4w5 sx/sp 487 // live laugh gerard...
4,1177133255881035782,LenaBaumgardt,Lena,Germany,1,104,"~ When things get hard, stop for awhile and lo..."


### Artist 2: Missy Elliott @MissyElliott

In [18]:
# note the artists' twitter handles
handles = ['MissyElliott']
handle_followers = []

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_obj = client.get_user(username=handle) # this will specify the handles
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    # this produces a print statement indicating artist
    print(f'Pulling followers for {handle}.')
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for followers in tweepy.Paginator(client.get_users_followers,
            user_obj.data.id,
            user_fields=["username","name","id","location","public_metrics","description"],
            max_results=1000,
            limit=100):
        print(followers.meta)
        
        for follower in followers.data:
            follower_row = {'id': follower.id,
                            'username': follower.username,
                            'name': follower.name,
                            'location': follower.location,
                            'follower_count': follower.public_metrics['followers_count'],
                            'following': follower.public_metrics['following_count'],
                            'description': follower.description
                           }
            user_data[handle].append(follower_row) # this appends all follower data
            followers_data[handle].append(follower.id) # this appends only follower ids
            
    # For each response object, extract the needed fields and store them in a dictionary or
    # data frame. 
    users_output_missy = pd.DataFrame(user_data[handle], columns=['id','username',
                                                               'name', 'location',
                                                               'follower_count', 'following',
                                                               'description'])
    followers_output_missy = pd.DataFrame(user_data[handle], columns=['id'])
    
    users_output_missy.to_csv(f'twitter/{handle}_followers.txt', sep='\t', index=False)
    followers_output_missy.to_csv(f'twitter/{handle}_follower_data.txt', sep='\t', index=False)
    
    # with help from Martin Zagari via slack, inserting planned sleep to help with rate limit        
    time.sleep(25+(10*random.random()))
            
    print(f'Completed pulling follower data for {handle}.')
        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(f'Time taken was {end_time - start_time}.')


Pulling followers for MissyElliott.
{'result_count': 1000, 'next_token': 'V7EQRGNJ9KPHGZZZ'}
{'result_count': 1000, 'next_token': 'O8UAJQ98TCP1GZZZ', 'previous_token': 'L7MP2V0PMB6EEZZZ'}
{'result_count': 1000, 'next_token': 'I4P3N1A7FSP1GZZZ', 'previous_token': 'OM2RVS7B2J6UEZZZ'}
{'result_count': 1000, 'next_token': 'O5IID5D620P1GZZZ', 'previous_token': '078PPMLSG36UEZZZ'}
{'result_count': 1000, 'next_token': 'AOTF2FNQP0OHGZZZ', 'previous_token': 'F0R9CK45TV6UEZZZ'}
{'result_count': 1000, 'next_token': 'AL2JUJUJBGOHGZZZ', 'previous_token': '2OBFU3OS6V7EEZZZ'}
{'result_count': 1000, 'next_token': 'UU6R7KDESKO1GZZZ', 'previous_token': '5TLS9EHQKF7EEZZZ'}
{'result_count': 1000, 'next_token': 'CL2HE0UBH4O1GZZZ', 'previous_token': 'P2QN2S2Q3B7UEZZZ'}
{'result_count': 1000, 'next_token': 'QRU57BFI2SO1GZZZ', 'previous_token': '681GGFPPER7UEZZZ'}
{'result_count': 1000, 'next_token': '2K2GDA14KONHGZZZ', 'previous_token': 'IQNCKI9KT37UEZZZ'}


Rate limit exceeded. Sleeping for 857 seconds.


{'result_count': 1000, 'next_token': '89NG2PVVAONHGZZZ', 'previous_token': '9AITJMO0BB8EEZZZ'}
{'result_count': 1000, 'next_token': '56DJJFCASSN1GZZZ', 'previous_token': 'SO6TPSO3L78EEZZZ'}
{'result_count': 1000, 'next_token': 'TRAPLF1MGKN1GZZZ', 'previous_token': '2D0447KS338UEZZZ'}
{'result_count': 1000, 'next_token': 'V14QM7RL88N1GZZZ', 'previous_token': 'CPJ2KH6AFB8UEZZZ'}
{'result_count': 1000, 'next_token': 'VERRM89BUGMHGZZZ', 'previous_token': '1OCA49CFNN8UEZZZ'}
{'result_count': 1000, 'next_token': 'PEBIF061E8MHGZZZ', 'previous_token': 'I809RR7A1F9EEZZZ'}
{'result_count': 998, 'next_token': 'E29BCDC75KMHGZZZ', 'previous_token': '6OFFCC2MHN9EEZZZ'}
{'result_count': 1000, 'next_token': 'LHFA31AS1KMHGZZZ', 'previous_token': 'LGSRQHSEQB9EEZZZ'}
{'result_count': 999, 'next_token': '31BAL9OUFSM1GZZZ', 'previous_token': '9VNK13D9UB9EEZZZ'}
{'result_count': 1000, 'next_token': 'TVK35ULHV0LHGZZZ', 'previous_token': '8CLP71NDG39UEZZZ'}
{'result_count': 1000, 'next_token': 'U13DDQPGEKLHGZ

Rate limit exceeded. Sleeping for 892 seconds.


{'result_count': 999, 'next_token': 'FB3R7H0GI8KHGZZZ', 'previous_token': 'TOE64FC81BBEEZZZ'}
{'result_count': 1000, 'next_token': 'I2GJLK9BSGK1GZZZ', 'previous_token': '5IHBKUNHDNBEEZZZ'}
{'result_count': 1000, 'next_token': 'LL9CU1PUBGK1GZZZ', 'previous_token': 'LDDVNDMN3FBUEZZZ'}
{'result_count': 1000, 'next_token': 'HAAQ27K5USJHGZZZ', 'previous_token': 'UCTJV4ENKFBUEZZZ'}
{'result_count': 999, 'next_token': '0HU8053QGOJHGZZZ', 'previous_token': 'DT0A82L613CEEZZZ'}
{'result_count': 1000, 'next_token': 'DFJJPR2ER4J1GZZZ', 'previous_token': 'O0GTRU6IF7CEEZZZ'}
{'result_count': 1000, 'next_token': 'OUBAMSSQ74J1GZZZ', 'previous_token': '2T0KCJ004VCUEZZZ'}
{'result_count': 1000, 'next_token': '5CMOS07HL8IHGZZZ', 'previous_token': '0J7A7LU3ORCUEZZZ'}
{'result_count': 1000, 'next_token': 'MFUTHJA9VKI1GZZZ', 'previous_token': 'G8DN82P1ANDEEZZZ'}
{'result_count': 999, 'next_token': 'HLLL9FTGF8I1GZZZ', 'previous_token': '91VE5SLP0BDUEZZZ'}
{'result_count': 1000, 'next_token': 'RAKQER1HS0HHGZZ

...

{'result_count': 1000, 'next_token': 'JPA0S7D0HSD1GZZZ', 'previous_token': 'OIG0F9QUQFIEEZZZ'}
{'result_count': 1000, 'next_token': 'VIJ0PNFKVOCHGZZZ', 'previous_token': 'E6GJQ4JAE3IUEZZZ'}
{'result_count': 1000, 'next_token': '9LA2UJ13BCCHGZZZ', 'previous_token': '4R9IM09307JEEZZZ'}
{'result_count': 1000, 'next_token': 'OCC19BEDQ4C1GZZZ', 'previous_token': '8E04761QKNJEEZZZ'}
{'result_count': 1000, 'next_token': 'PEPCNQTN6GC1GZZZ', 'previous_token': '8DEBC1PL5RJUEZZZ'}
{'result_count': 1000, 'next_token': 'LM7R1H71O8BHGZZZ', 'previous_token': 'FDTKI62TPFJUEZZZ'}
{'result_count': 1000, 'next_token': 'CT4L89318GBHGZZZ', 'previous_token': 'GEAOTQOU7NKEEZZZ'}
{'result_count': 1000, 'next_token': 'DQ7QTR40FCB1GZZZ', 'previous_token': 'S4EELKD3NFKEEZZZ'}
{'result_count': 1000, 'next_token': '0J4MF6KEMKAHGZZZ', 'previous_token': 'I6DTTRKPGJKUEZZZ'}
{'result_count': 1000, 'next_token': 'CJUVQKPMT0A1GZZZ', 'previous_token': '6CJDP74Q9BLEEZZZ'}
{'result_count': 1000, 'next_token': 'EANOU06THCA1

Rate limit exceeded. Sleeping for 892 seconds.


{'result_count': 1000, 'next_token': '8CH4HOLO4K91GZZZ', 'previous_token': 'CJ3IP8LO9NMUEZZZ'}
{'result_count': 1000, 'next_token': 'D0D3B37MDG8HGZZZ', 'previous_token': '632GU5IHRBMUEZZZ'}
{'result_count': 1000, 'next_token': 'MSFRV03MJ481GZZZ', 'previous_token': 'B6BARAH9IFNEEZZZ'}
{'result_count': 1000, 'next_token': 'J3UE8NKBUS7HGZZZ', 'previous_token': 'SP8B5UTECRNUEZZZ'}
{'result_count': 1000, 'next_token': 'S5VACRQECK7HGZZZ', 'previous_token': 'G3TKG6CO13OEEZZZ'}
{'result_count': 1000, 'next_token': 'EE2RJMEGKK71GZZZ', 'previous_token': '7DMSVK7MJBOEEZZZ'}
{'result_count': 1000, 'next_token': 'JR0AF60GSG6HGZZZ', 'previous_token': 'RLD81EQNBBOUEZZZ'}
{'result_count': 1000, 'next_token': 'MIFBGDUN4C6HGZZZ', 'previous_token': 'R34LSP8O3JPEEZZZ'}
{'result_count': 1000, 'next_token': 'SID9SJUCJ061GZZZ', 'previous_token': 'BH8JEDRVRJPEEZZZ'}
{'result_count': 1000, 'next_token': '904E6DI62461GZZZ', 'previous_token': 'TG1D5620CVPUEZZZ'}
{'result_count': 1000, 'next_token': 'BN2L9B2HCC5H

Rate limit exceeded. Sleeping for 892 seconds.


{'result_count': 1000, 'next_token': 'MDALOLLSN041GZZZ', 'previous_token': 'RVK5F5L7OFREEZZZ'}
{'result_count': 1000, 'next_token': 'SM7O4AD83O41GZZZ', 'previous_token': 'L2D2VB2M8VRUEZZZ'}
{'result_count': 1000, 'next_token': 'JK5M9K08JK3HGZZZ', 'previous_token': 'UN911KB1S7RUEZZZ'}
{'result_count': 1000, 'next_token': '26GG9BNL5O3HGZZZ', 'previous_token': 'C9ENFTAICFSEEZZZ'}
{'result_count': 1000, 'next_token': 'NBIA7AJORG31GZZZ', 'previous_token': 'KDPE3HOKQ7SEEZZZ'}
{'result_count': 1000, 'next_token': 'I6P9QJE7GG31GZZZ', 'previous_token': 'L95AV4S94FSUEZZZ'}
{'result_count': 1000, 'next_token': 'EIRG6KFF5431GZZZ', 'previous_token': 'O04D1B21FFSUEZZZ'}
{'result_count': 1000, 'next_token': 'CAV0LJ0FH42HGZZZ', 'previous_token': 'I8P4BA0JQRSUEZZZ'}
{'result_count': 1000, 'next_token': '0509CUB54C2HGZZZ', 'previous_token': 'D9RUC7BMEVTEEZZZ'}
{'result_count': 1000, 'next_token': 'U59CB7BQOS21GZZZ', 'previous_token': 'PK6BL0M2RJTEEZZZ'}
{'result_count': 1000, 'next_token': 'OIVO7OOIH021

In [19]:
# checking dataframes to ensure proper data pull
users_output_missy.head()

,id,username,name,location,follower_count,following,description
0,1531138365478903808,KevJay20_,Kev Jay,None,0,44,
1,1340011142077423621,NtandokayiseTE1,@TIDO-FIGO,"Gauteng , South Africa",688,4670,I left earth🌍🌎🌏🗺️🧭 a while ago🌊...
2,1569955923426156544,benjamin6_joram,Joram Benjamin,None,1,121,
3,1569962253570523137,Cassandralive50,Cassandra Jackson,None,1,42,
4,1219417315030245377,Geecabin,Gee Jones,None,1,106,


In [20]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description

'Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well.'

# Lyrics Scrape

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [21]:
artists = {'robyn':"https://www.azlyrics.com/m/mychemicalromance.html",
           'cher':"https://www.azlyrics.com/m/missy.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> 


In [22]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

Let's make sure we have enough lyrics pages to scrape. 

In [23]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [24]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [25]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [26]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [27]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    # 2. Iterate over the lyrics pages
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
    # 4. Extract the title and lyrics from the page.
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

SyntaxError: unexpected EOF while parsing (<ipython-input-27-9e7b4f75ca4d>, line 20)

In [ ]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [ ]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [ ]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

In [ ]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [ ]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")
